# Booming Asset Detection

Early warning dashboard for assets transitioning into breakout regimes.

## TODO
- [ ] Define breakout labels using rolling returns, volatility contraction, and liquidity expansion thresholds.
- [ ] Fuse social/on-chain signals with market data to build composite alert scores.
- [ ] Validate precision and recall with walk-forward tests across multiple regimes.


In [ ]:
# %pip install --quiet quantconnect-stubs
from datetime import datetime
from datetime import timedelta
from pathlib import Path
from typing import Iterable

import pandas as pd
from research.scripts.data_loader import DataLoader, DataRequestSpec

pd.options.display.max_columns = 20

try:
    from QuantConnect import Resolution  # type: ignore[import]
    from QuantConnect.Research import QuantBook  # type: ignore[import]
except ImportError as exc:  # pragma: no cover - requires QuantConnect runtime
    raise RuntimeError(
        "QuantConnect Research assemblies not found. Run inside a Lean Research environment."
    ) from exc

qb = QuantBook()


def enum_to_str(enum_obj) -> str:
    """Return a string representation of a QuantConnect enum."""

    if hasattr(enum_obj, "ToString"):
        return enum_obj.ToString()
    name = getattr(enum_obj, "name", None)
    if name is not None:
        return name
    return str(enum_obj)


UNIVERSE_ROOT = Path("data/selection")
UNIVERSE_ROOT.mkdir(parents=True, exist_ok=True)

DEFAULT_TICKERS = ["BTCUSD", "ETHUSD", "SOLUSD", "DOGEUSD"]


def add_selection_assets(tickers: Iterable[str], resolution: Resolution = Resolution.Hour):
    """Attach breakout candidate assets to the research QuantBook."""

    symbols = []
    for ticker in tickers:
        symbol = qb.AddCrypto(ticker, resolution).Symbol
        symbols.append(symbol)
    return symbols


DEFAULT_RESOLUTION = Resolution.Hour
SYMBOLS = add_selection_assets(DEFAULT_TICKERS, DEFAULT_RESOLUTION)

REQUEST_START = datetime(2023, 1, 1)
REQUEST_END = datetime.utcnow()
LOOKBACK = timedelta(days=120)

loader = DataLoader()
for symbol in SYMBOLS:
    loader.register(
        DataRequestSpec(
            symbol=symbol.Value,
            market=symbol.ID.Market,
            security_type="Crypto",
            resolution=enum_to_str(DEFAULT_RESOLUTION),
            start=REQUEST_START.isoformat(),
            end=REQUEST_END.isoformat(),
            additional_params={"lookback": LOOKBACK.days},
        )
    )


In [ ]:
RUN_BREAKOUT_SAMPLE = False

if RUN_BREAKOUT_SAMPLE:
    history = qb.History(SYMBOLS, LOOKBACK, DEFAULT_RESOLUTION)
    closes = history.close.unstack(level=0).tz_localize(None)
    hourly_returns = closes.pct_change().dropna()
    hourly_returns.tail()
